# Working with the OpenAI API - Part 3

## Going Beyond Text Completions

OpenAI has not only developed models for text generation but also for text moderation and audio transcription and translation. You'll learn to use OpenAI's moderation models to detect violations of their terms of use—a crucial function in user-facing applications. You'll also discover how the Whisper model can be used to transcribe and translate audio from different languages, which has huge applications in automated meeting notes and caption generation.

In [3]:
import os

api_key = os.getenv('OPENAI_API_KEY')

In [4]:
from openai import OpenAI

# Create the OpenAI client
client = OpenAI(api_key=api_key)

### Requesting moderation
Aside from text and chat completion models, OpenAI provides models with other capabilities, including text moderation. OpenAI's text moderation model is designed for evaluating prompts and responses to determine if they violate OpenAI's usage policies, including inciting hate speech and promoting violence.

In this exercise, you'll test out OpenAI's moderation functionality on a sentence that may have been flagged as containing violent content using traditional word detection algorithms.

In [5]:
# Create a request to the Moderation endpoint
response = client.moderations.create(
    model="text-moderation-latest",
    input="My favorite book is To Kill a Mockingbird."
)

# Print the category scores
print(response.results[0].category_scores)

CategoryScores(harassment=5.243551186140394e-06, harassment_threatening=1.1516095810293336e-06, hate=4.767837526742369e-05, hate_threatening=3.2021056028952444e-08, illicit=None, illicit_violent=None, self_harm=9.466615438213921e-07, self_harm_instructions=5.426785065765216e-08, self_harm_intent=1.5536235764557205e-07, sexual=3.545879735611379e-06, sexual_minors=1.1304399549771915e-06, violence=0.0001064608441083692, violence_graphic=1.086988686438417e-05, self-harm=9.466615438213921e-07, sexual/minors=1.1304399549771915e-06, hate/threatening=3.2021056028952444e-08, violence/graphic=1.086988686438417e-05, self-harm/intent=1.5536235764557205e-07, self-harm/instructions=5.426785065765216e-08, harassment/threatening=1.1516095810293336e-06)


### Examining moderation category scores
The same request you created in the last exercise to the Moderation endpoint has been run again, sending the sentence "My favorite book is To Kill a Mockingbird." to the model. The response from the API has been printed for you, and is available as response.

What is the correct interpretation of the category_scores here?

**Answer:** The model believes that there are no violations, as all categories are close to 0.

### Creating a podcast transcript
The OpenAI API Audio endpoint provides access to the Whisper model, which can be used for speech-to-text transcription and translation. In this exercise, you'll create a transcript from a DataFramed podcast episode with OpenAI Developer, Logan Kilpatrick.

If you'd like to hear more from Logan, check out the full ChatGPT and the OpenAI Developer Ecosystem podcast episode.

In [7]:
# Open the openai-audio.mp3 file
audio_file = open("openai-audio.mp3", "rb")

# Create a transcript from the audio file
response = client.audio.transcriptions.create(model="whisper-1", file=audio_file)

# Extract and print the transcript text
print(response.text)

Hi there, Logan, thank you for joining us on the show today. Thanks for having me. I'm super excited about this. Brilliant. We're going to dive right in, and I think ChatGPT is maybe the most famous AI product that you have at OpenAI, but I'd just like to get an overview of what all the other AIs that are available are. So I think two and a half years ago, OpenAI released the API that we still have available today, which is essentially our giving people access to these models. And for a lot of people, giving people access to the model that powers ChatGPT, which is our consumer-facing first-party application, which essentially just, in very simple terms, puts a nice UI on top of what was already available through our API for the last two and a half years. So it's sort of democratizing the access to this technology through our API. If you want to just play around with it, as an end user, we have ChatGPT available to the world as well.


### Transcribing a non-English language
The Whisper model can not only transcribe English language, but also performs well on speech in many other languages.

In this exercise, you’ll create a transcript from audio.m4a, which contains speech in Portuguese.

In [9]:
# Open the audio.m4a file
audio_file= open("audio.m4a", "rb")

# Create a transcript from the audio file
response = client.audio.transcriptions.create(model="whisper-1", file=audio_file)

print(response.text)

Olá, o meu nome é Eduardo, sou CTO no Datacamp. Espero que esteja a gostar deste curso que o James e eu criamos para você. Esta API permite enviar um áudio e trazer para inglês. O áudio original está em português.


### Translating Portuguese
Whisper can not only transcribe audio into its native language but also supports translation capabilities for creating English transcriptions.

In this exercise, you'll return to the Portuguese audio, but this time, you'll translate it into English!

In [10]:
# Open the audio.m4a file
audio_file = open("audio.m4a", "rb")

# Create a translation from the audio file
response = client.audio.translations.create(model="whisper-1", file=audio_file)

# Extract and print the translated text
print(response.text)

Hello, my name is Eduardo, I am a CTO at Datacamp. I hope you are enjoying this course that James and I have created for you. This API allows you to send an audio and bring it to English. The original audio is in Portuguese.


### Translating with prompts
The quality of Whisper's translation can vary depending on the language spoken, the audio quality, and the model's awareness of the subject matter. If you have any extra context about what is being spoken about, you can send it along with the audio to the model to give it a helping hand.

You've been provided with with an audio file, audio.wav; you're not sure what language is spoken in it, but you do know it relates to a recent World Bank report. Because you don't know how well the model will perform on this unknown language, you opt to send the model this extra context to steer it in the right direction.

In [ ]:
# Open the audio.wav file
audio_file = open("audio.wav", "rb")

# Write an appropriate prompt to help the model
prompt = "The audio relates to a recent World Bank report."

# Create a translation from the audio file
response = client.audio.translations.create(
    model="whisper-1", 
    file=audio_file,
    prompt=prompt)

print(response.text)

### Identifying audio language
You've learned that you're not only limited to creating a single request, and that you can actually feed the output of one model as an input to another! This is called chaining, and it opens to the doors to more complex, multi-modal use cases.

In this exercise, you'll practice model chaining to identify the language used in an audio file. You'll do this by bringing together OpenAI's audio transcription functionality and its text models with only a few lines of code.

In [11]:
# Open the audio.wav file
audio_file = open("audio.m4a", "rb")

# Create a transcription request using audio_file
audio_response = client.audio.transcriptions.create(
    model="whisper-1", 
    file=audio_file
)

transcript = audio_response.text
prompt = "Identify the language of the following text:\n\n" + transcript

# Create a request to the API to identify the language spoken
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[{"role": "user", 
               "content": prompt}],
    max_tokens=10
)

chat_response = response.choices[0].message.content
print(chat_response)

The language of the text is Portuguese.


### Creating meeting summaries
Time for business! One time-consuming task that many find themselves doing day-to-day is taking meeting notes to summarize attendees, discussion points, next steps, etc.

In this exercise, you'll use AI to augment this task to not only save a substantial amount of time, but also to empower attendees to focus on the discussion rather than administrative tasks. You've been provided with a recording from DataCamp's Q2 Roadmap webinar, which summarizes what DataCamp will be releasing during that quarter. You'll chain the Whisper model with a text or chat model to discover which courses will be launched in Q2.

In [13]:
# Open the datacamp-q2-roadmap.mp3 file
audio_file = open("openai-audio.mp3", "rb")

# Create a transcription request using audio_file
audio_response = client.audio.transcriptions.create(
    model="whisper-1",
    file=audio_file
)

# Create a request to the API to summarize the transcript into bullet points
transcript = audio_response.text
prompt = "Summarize following text into concise bullet points: " + transcript

response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[{"role": "user", 
               "content": prompt}],
)

chat_response = response.choices[0].message.content
print(chat_response)

- Logan expressed excitement about being on the show.
- Overview requested on OpenAI's AI products beyond ChatGPT.
- OpenAI released its API 2.5 years ago, providing access to AI models.
- ChatGPT serves as a consumer-facing application, utilizing the models from the API.
- ChatGPT offers a user-friendly interface, making the technology accessible.
- The goal is to democratize access to AI technology through both the API and ChatGPT.
